In [1]:
from models.codec.kmeans.kmeans_model import KMeans, KMeansEMA
from transformers import Wav2Vec2BertModel
from transformers import SeamlessM4TFeatureExtractor
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import librosa
from IPython.display import Audio
import matplotlib.pyplot as plt
import safetensors

In [2]:
kmeans_model = KMeansEMA(2048, 1024)
# model_path = "/blob/v-yuancwang/codec_ckpt/semantic_kmeans/semantic_kmeans_ema_debug_libritts/checkpoint/epoch-0010_step-0002000_loss-679.313721/pytorch_model.bin"
# kmeans_model.load_state_dict(torch.load(model_path))
model_path = "/blob/v-yuancwang/codec_ckpt/semantic_kmeans/semantic_kmeans_ema_debug_emilia_1k_2048_2/model.safetensors"
safetensors.torch.load_model(kmeans_model, model_path)
kmeans_model.eval()
kmeans_model.cuda()

KMeansEMA(
  (dumy_emb): Embedding(1, 1)
)

In [3]:
wav_path_0 = "/home/t-zeqianju/yuancwang/temp_test_dataset/libritts-train-clean-100/train-clean-100/8468/294887/8468_294887_000017_000002.wav"
# wav_path_1 = "/home/t-zeqianju/yuancwang/temp_test_dataset/libritts-train-clean-100/train-clean-100/5022/29405/5022_29405_000025_000003.wav"
# wav_path_2 = "/home/t-zeqianju/yuancwang/temp_test_dataset/libritts-train-clean-100/train-clean-100/5022/29411/5022_29411_000051_000002.wav"
# wav_path_4 = "/home/t-zeqianju/yuancwang/temp_test_dataset/libritts-train-clean-100/train-clean-100/1502/122619/1502_122619_000052_000002.wav"
# # concat 4 wav
wav_0, sr = librosa.load(wav_path_0, sr=16000)
# wav_1, sr = librosa.load(wav_path_1, sr=16000)
# wav_2, sr = librosa.load(wav_path_2, sr=16000)
# wav_4, sr = librosa.load(wav_path_4, sr=16000)
# wav = np.concatenate([wav_0, wav_1, wav_2, wav_4])
wav = wav_0
Audio(wav, rate=16000)

In [4]:
processor = SeamlessM4TFeatureExtractor.from_pretrained("facebook/w2v-bert-2.0")
semantic_model = Wav2Vec2BertModel.from_pretrained("facebook/w2v-bert-2.0")
semantic_model.eval()
semantic_model = semantic_model.cuda()
layer_idx = 15
if layer_idx == 23:
    output_idx = 0
else:
    output_idx = layer_idx + 2
stat_mean_var = torch.load("/home/t-zeqianju/yuancwang/mls_wav2vec2bert_stats.pt")
semantic_mean = stat_mean_var["mean"]
semantic_std = torch.sqrt(stat_mean_var["var"])
semantic_mean = semantic_mean.to("cuda")
semantic_std = semantic_std.to("cuda")
print("semantic mean: ", semantic_mean, "semantic std: ", semantic_std)

semantic mean:  tensor([-0.1368, -0.1855,  0.2000,  ...,  0.0310,  0.1405, -0.1794],
       device='cuda:0', dtype=torch.float64) semantic std:  tensor([0.3364, 0.4122, 0.3472,  ..., 0.3680, 0.3667, 0.3582], device='cuda:0',
       dtype=torch.float64)


In [5]:
speech_data = wav
inputs = processor(speech_data, sampll_rate=16000)
input_features = inputs["input_features"]
print(input_features.shape)
attention_mask = inputs["attention_mask"]
print(attention_mask.shape)
# print(attention_mask)

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


(1, 589, 160)
(1, 589)


In [6]:
input_features = torch.tensor(input_features).cuda()
attention_mask = torch.tensor(attention_mask).cuda()

In [7]:
with torch.no_grad():
    vq_emb = semantic_model(
        input_features=input_features,
        attention_mask=attention_mask,
        output_hidden_states=True,
    )
    feat = vq_emb.hidden_states[output_idx]  # B x T x C
    feat = (feat - semantic_mean.to(feat)) / semantic_std.to(feat)

In [8]:
vq_id, _ = kmeans_model.quantize(feat)
vq_id = vq_id.cpu().numpy()[0]
print(vq_id)

[ 295  614 1751 1448  750 1961 1511 1271  200 1056 1770  169 1953 1315
 2037  597  349    8 1543 1543  405  351 1742  415  437 1474  199  602
 1101  830 1181 1155 1155  596 1145 1077 1823  940  151  980  980  270
  270 1037 1488 1121 1972  905  905 2012 1801 1827  597 1170 1170  315
  209  209 1551  521 1533  575  521  538  367 1830  152  614  976  155
  510  609  842  965  503 1138 1131 1527 1875 1989 1563 1191 1772 1145
 1077  990 1958  302 1022  519  749  519  873 2044 2044 1989  907  694
 1197 1842  155  510 1846  405  351 1742 1920   28 2011 1148   59   97
   36 1067 1506    1 1274 1191 1650 1641 1546  935   26 1772   23  136
 1923  328  990   28  975 1306  152   59  279   97  429  350 1295  350
 1295 1084 1084 1084 1771 1664 1265  814  949  925  925 1636  236  341
 1975 1904 1252 1220 1148  190  244  763 1333 1989 1563    1 1672 1503
 1908  368  778  778  844  844 1335 1335 1458 1636  236 1743  761  645
 1360 1144 1005  283 2028 2028   27 1050  873 1621 1315 2037 1315  896
  896 

In [9]:
unique, counts = np.unique(vq_id, return_counts=True)
idx = np.argsort(counts)[::-1]
unique = unique[idx]
counts = counts[idx]
print(unique)

[1395 1121 1707 1975  431  503  769  226 1495  152  405  842 1693  167
 1621 1145  150 1315 1247 1252 1155  965 1360 1077  527  534 1812  575
 1418  737 1274  800 1265 1084 1551 1672 1231  896  905  521  614  121
   88  157  209 1972 1989  306  315  176  350  363  351    7  137 1067
 1068 1072  163 1377 1371  155 1111 1148 1170 1295 1181 1191 1220 1050
  102    6   97 1335   59   28  508  976  990  302  519  457  538  347
  597  598  609  341  727  749  778  798  836  980  270  257  245  844
  236  847  848  873  199  925  189  510 2044    1 1637 1991 1846 1733
 1488 2010 1503 1569 1563 1506 1928 2015 1963 1961 1959 1659 1543 1533
 1875 1670 1742 2011 1456 1899 2037 1772 1636 2028 1913 1823 1814 1942
 1922  349 1904  425  423  429  391 1902  367  417 1886 1900  353  402
  388  368  415 1901 1419  437 1850 1800  596  576 1801  568  558  544
 1821 1827 1830  511 1840 1842  507  504  502  494  492  489  488  338
  481  459  484  283  329  136  118  108  104  103 2012   95   80   76
   70 

In [10]:
idx = np.where(vq_id == 800)[0]
print(idx)

[462 463 464]


In [11]:
# concat
cat_wav = np.array([])
for i in range(0, len(idx)):
    cat_wav = np.concatenate([cat_wav, wav[(idx[i]-5) * 320: (idx[i] + 5) * 320]])
    cat_wav = np.concatenate([cat_wav, np.zeros(16000)])
Audio(cat_wav, rate=16000)

# ABX test

In [12]:
test_path = "/home/t-zeqianju/yuancwang/LibriSpeech/dev-clean"
os.makedirs("/home/t-zeqianju/yuancwang/ABX_features/dev-clean/kmeans_emilia_1k_2048_debug_2", exist_ok=True)
for spk in os.listdir(test_path):
    for chapter in os.listdir(os.path.join(test_path, spk)):
        for wav_file in os.listdir(os.path.join(test_path, spk, chapter)):
            if not wav_file.endswith(".flac"):
                continue
            wav_path = os.path.join(test_path, spk, chapter, wav_file)
            wav, sr = librosa.load(wav_path, sr=16000)
            inputs = processor(wav, sampling_rate=16000)
            input_features = inputs["input_features"]
            attention_mask = inputs["attention_mask"]
            input_features = torch.tensor(input_features).cuda()
            attention_mask = torch.tensor(attention_mask).cuda()
            with torch.no_grad():
                vq_emb = semantic_model(
                    input_features=input_features,
                    attention_mask=attention_mask,
                    output_hidden_states=True,
                )
                feat = vq_emb.hidden_states[output_idx]
                feat = (feat - semantic_mean.to(feat)) / semantic_std.to(feat)
                # print(feat.shape)
                quantize, _, _ = kmeans_model(feat)
                # print(quantize.shape)
                uid = wav_file.split(".")[0]
                # print(uid)
                quantize = quantize.cpu().numpy()[0]
                save_path = os.path.join("/home/t-zeqianju/yuancwang/ABX_features/dev-clean/kmeans_emilia_1k_2048_debug_2", uid + ".npy")
                np.save(save_path, quantize)